In [2]:
import pandas as pd
import json
import requests
from googleapiclient.discovery import build
from IPython.display import JSON
from config.config import YOUTUBE_API_KEY

In [18]:
youtube = build('youtube', 'v3', developerKey = YOUTUBE_API_KEY)

In [19]:
# Seçtiğimiz video'nun id si.
video_id = 'VgoyV_ghAdo'


In [20]:
# Video istatistiklerini almak için API isteği oluşturalım.
request = youtube.videos().list(
    part="snippet,statistics",
    id=video_id
)
response = request.execute()

# JSON formatında çıktıyı yazdıralım.
print(json.dumps(response, indent=2))

{
  "kind": "youtube#videoListResponse",
  "etag": "_6omuwDAUfTGLXrgo0cZkrpWr7M",
  "items": [
    {
      "kind": "youtube#video",
      "etag": "nceybl7KSFJiutHs3jGmsUkxgHk",
      "id": "VgoyV_ghAdo",
      "snippet": {
        "publishedAt": "2022-10-07T14:30:12Z",
        "channelId": "UCdVBWUBCuREx1Q2Ikw9R8Mw",
        "title": "Uzay\u0131n Bilinmeyenleri | Zaman\u0131 Yakalamak | TRT Belgesel",
        "description": "Uzay\u0131n Bilinmeyenleri\u2019nin bu b\u00f6l\u00fcm\u00fcnde, 12 bin y\u0131l \u00f6ncesinde G\u00f6beklitepe\u2019yi in\u015fa eden insanlardan bu yana evrenle ilgili ke\u015ffetti\u011fimiz detaylar\u0131 ve bu s\u00fcre\u00e7te  zaman\u0131 \u00f6l\u00e7me y\u00f6ntemlerinin nas\u0131l de\u011fi\u015fti\u011fini inceliyoruz.\n\nZaman\u0131 \u00f6l\u00e7me yolculu\u011fumuza tarihin s\u0131f\u0131r noktas\u0131 olarak bilinen G\u00f6beklitepe\u2019de ba\u015fl\u0131yoruz. \u0130lk insanlar\u0131n ay ve mevsim kavramlar\u0131n\u0131 nas\u0131l ke\u015ffetti\u01

In [21]:
video_info = response['items'][0]
title = video_info['snippet']['title']
published_at = video_info['snippet']['publishedAt']
view_count = video_info['statistics']['viewCount']
like_count = video_info['statistics']['likeCount']
favorite_count = video_info['statistics']['favoriteCount']
comment_count = video_info['statistics']['commentCount']

# Verileri DataFrame'e çevirme
data = {
    "Video Adı": [title],
    "Yayın Tarihi": [published_at],
    "Görüntülenme Sayısı": [view_count],
    "Beğeni Sayısı": [like_count],
    "Favorilere Ekleme Sayısı": [favorite_count],
    "Yorum Sayısı": [comment_count]
}

video_df = pd.DataFrame(data)

In [32]:
video_df.head()

,Video Adı,Yayın Tarihi,Görüntülenme Sayısı,Beğeni Sayısı,Favorilere Ekleme Sayısı,Yorum Sayısı
0,Uzayın Bilinmeyenleri | Zamanı Yakalamak | TRT...,2022-10-07T14:30:12Z,151510,1888,0,89


# Video hakkında yapılan yorumları inceliyoruz.

In [23]:
request = youtube.commentThreads().list(
    part='snippet',
    videoId= video_id,
    maxResults=100  # maximum 100 yorum alabiliyoruz
)
response = request.execute()

comment = [item['snippet']['topLevelComment']['snippet']['textDisplay'] for item in response['items']]
for item in comment:
    print(item)

Tüm dünya iki şeyin kıymetini çok iyi öğrenecek Beyaz onlukleriyle BILIM INSANLARI  Çamurlu cizmeleriyle CIFTCILER.
Bütün vidio larını izledim düzceden slm
👏👏👏
Yani bunlarin içinde ve dışında yani uzayda ay mars buralarda bizim Türkiye nin araştırma yapması lazim
Haydi Türkiye m kurban olam daha ileri dünya da söz sahibi olalım halkımız refah ve tüm insanlık için adaleti türkler qetirsin
Yayınlarınız çok ilgi çekici🎉
cosmos belgeselinden bile kalitelisin be trt belgesel harikasınız
Bazen cok yalniz hissediyorum bu dunyada..herkes bu kadar aptal mi diye dusunmeden edemiyorum..birakin salak batinin salak tezlerini, dinsiz teorilerini..boyle bir yere varamazsiniz anlayin artik..hic bir seyin cevabini veremiyorsunuz..hersey bir gizem..bazilari daha merakliymis da, goge bakmis da, yildizlari gormus, gece gunduzu farketmismis..tobe estagfirullah..yazik ya, yazik..
Verdiğim Vergiler Helal Olsun ❤
Hoca böyle anlattı da biz mi öğrenmedik.
Uzay ve zamanı  yakalamak IMANLA mümkündür  ,yoksa bir m

In [24]:
# videonun hangi kanala ait olduğunu, yorumun kime ait olduğunu, yoruma gelen beğeni ve yorum sayılarını bulmak için liste oluşturalım.
comments_list = []
for item in response['items']:
    snippet = item['snippet']['topLevelComment']['snippet']
    comment_dict = {
        'ChannelId': snippet['authorChannelId']['value'],
        'AuthorDisplayName': snippet['authorDisplayName'],
        'CommentText': snippet['textDisplay'],
        'LikeCount': snippet['likeCount'],
        'ReplyCount': item['snippet']['totalReplyCount']
    }
    comments_list.append(comment_dict)

# Liste içindeki verileri DataFrame'e çevirin
comments_df = pd.DataFrame(comments_list)

In [25]:
comments_df

,ChannelId,AuthorDisplayName,CommentText,LikeCount,ReplyCount
0,UCYJGD84GDbPUwy3HQx1b0EA,@aysekalyoncu986,Tüm dünya iki şeyin kıymetini çok iyi öğrenece...,0,0
1,UCGdo2iH8LUL4Z1skVbojO9g,@karadenizsolucankrallg6943,Bütün vidio larını izledim düzceden slm,1,0
2,UC3hq_UkBLMZiQpwTsfQlfjA,@caglartasl1450,👏👏👏,1,0
3,UCiiU6tGmmg3t4-dkSKya4Kg,@Mert-vf1qe,Yani bunlarin içinde ve dışında yani uzayda ay...,0,0
4,UCiiU6tGmmg3t4-dkSKya4Kg,@Mert-vf1qe,Haydi Türkiye m kurban olam daha ileri dünya d...,1,0
...,...,...,...,...,...
73,UC6RVXHVFDDYxhX8Vw9aCxYQ,@nebula9123,TRT türk düşmanı diziler yaparken TRT belgesel...,3,0
74,UCqzPIuieynMAjXs1Rc2_obA,@emineatmaca8283,Harika olmuş emeğine yüreğine sağlık bu güzel ...,11,0
75,UCYTNl79yrRKBVdh1eNMxBxw,@onursolmaz4491,eskiden videolar fragman şeklindeydi TRT uygul...,1,0
76,UCE_MMo9uYBeKBc7Z8PcwQCw,@D1Y4R.,Tiktok videolarının daha çok izlenmesi gerçeği :(,5,1


In [26]:
# Yorum, beğeni sayısı, yoruma yapılan yorum sayıları
select_columns = ['CommentText', 'LikeCount', 'ReplyCount']
select_data = comments_df[select_columns]

select_data_df = pd.DataFrame(select_data)
select_data_df

,CommentText,LikeCount,ReplyCount
0,Tüm dünya iki şeyin kıymetini çok iyi öğrenece...,0,0
1,Bütün vidio larını izledim düzceden slm,1,0
2,👏👏👏,1,0
3,Yani bunlarin içinde ve dışında yani uzayda ay...,0,0
4,Haydi Türkiye m kurban olam daha ileri dünya d...,1,0
...,...,...,...
73,TRT türk düşmanı diziler yaparken TRT belgesel...,3,0
74,Harika olmuş emeğine yüreğine sağlık bu güzel ...,11,0
75,eskiden videolar fragman şeklindeydi TRT uygul...,1,0
76,Tiktok videolarının daha çok izlenmesi gerçeği :(,5,1


In [27]:
# ilk 5 satırı okutalım
select_data_df.head(5)

,CommentText,LikeCount,ReplyCount
0,Tüm dünya iki şeyin kıymetini çok iyi öğrenece...,0,0
1,Bütün vidio larını izledim düzceden slm,1,0
2,👏👏👏,1,0
3,Yani bunlarin içinde ve dışında yani uzayda ay...,0,0
4,Haydi Türkiye m kurban olam daha ileri dünya d...,1,0


In [28]:
# son 5 satırı okutalım
select_data_df.tail()

,CommentText,LikeCount,ReplyCount
73,TRT türk düşmanı diziler yaparken TRT belgesel...,3,0
74,Harika olmuş emeğine yüreğine sağlık bu güzel ...,11,0
75,eskiden videolar fragman şeklindeydi TRT uygul...,1,0
76,Tiktok videolarının daha çok izlenmesi gerçeği :(,5,1
77,Daha fazla uzayın bilinmeyenleri gelsin,4,0


In [29]:
# Maksimum beğeni alan yorumu bulma
max_like_comment = comments_df.loc[comments_df['LikeCount'].idxmax()]

# Maksimum yorum alan yorumu bulma
max_reply_comment = comments_df.loc[comments_df['ReplyCount'].idxmax()]

result_df = pd.DataFrame([max_like_comment, max_reply_comment])
result_df

,ChannelId,AuthorDisplayName,CommentText,LikeCount,ReplyCount
67,UCfY3QQmFN0AxmP7vUwFRyHw,@DARK-LORD-12,Dünyanın önde gelen bütün belgeselleri şu TRT ...,56,5
67,UCfY3QQmFN0AxmP7vUwFRyHw,@DARK-LORD-12,Dünyanın önde gelen bütün belgeselleri şu TRT ...,56,5


In [30]:
import pandas as pd

request = youtube.commentThreads().list(
    part='snippet',
    videoId=video_id,
    maxResults=100
)
response = request.execute()

# Her yorumu temsil eden bir sözlük oluşturun ve bu sözlükleri bir listeye ekleyin
comments_list = []
for item in response['items']:
    snippet = item['snippet']['topLevelComment']['snippet']
    comment_dict = {
        'ChannelId': snippet['authorChannelId']['value'],
        'AuthorDisplayName': snippet['authorDisplayName'],
        'CommentText': snippet['textDisplay'],
        'LikeCount': snippet['likeCount']
    }
    comments_list.append(comment_dict)

# Liste içindeki verileri DataFrame'e çevirin
comments_df = pd.DataFrame(comments_list)


In [31]:
comments_df

,ChannelId,AuthorDisplayName,CommentText,LikeCount
0,UCYJGD84GDbPUwy3HQx1b0EA,@aysekalyoncu986,Tüm dünya iki şeyin kıymetini çok iyi öğrenece...,0
1,UCGdo2iH8LUL4Z1skVbojO9g,@karadenizsolucankrallg6943,Bütün vidio larını izledim düzceden slm,1
2,UC3hq_UkBLMZiQpwTsfQlfjA,@caglartasl1450,👏👏👏,1
3,UCiiU6tGmmg3t4-dkSKya4Kg,@Mert-vf1qe,Yani bunlarin içinde ve dışında yani uzayda ay...,0
4,UCiiU6tGmmg3t4-dkSKya4Kg,@Mert-vf1qe,Haydi Türkiye m kurban olam daha ileri dünya d...,1
...,...,...,...,...
73,UC6RVXHVFDDYxhX8Vw9aCxYQ,@nebula9123,TRT türk düşmanı diziler yaparken TRT belgesel...,3
74,UCqzPIuieynMAjXs1Rc2_obA,@emineatmaca8283,Harika olmuş emeğine yüreğine sağlık bu güzel ...,11
75,UCYTNl79yrRKBVdh1eNMxBxw,@onursolmaz4491,eskiden videolar fragman şeklindeydi TRT uygul...,1
76,UCE_MMo9uYBeKBc7Z8PcwQCw,@D1Y4R.,Tiktok videolarının daha çok izlenmesi gerçeği :(,5


In [1]:
from googletrans import Translator

In [2]:
translator = Translator(service_url = ['translate.google.com'])
ifade = "İlk saniyelerdeki Van detayı beni oldukça mutlu etti."
ceviri = translator.translate(ifade)
ceviri.text

TypeError: __init__() got an unexpected keyword argument 'service_url'